# 如何将自定义系统提示添加到预构建的 ReAct 代理中

本教程将展示如何向预构建的 ReAct 代理添加自定义系统提示。请参阅 [本教程](./create-react-agent.ipynb) 了解如何开始使用预构建的 ReAct 代理

您可以通过将字符串传递给 `stateModifier` 参数来添加自定义系统提示。

<div class="警告提示">
<p class="admonition-title">兼容性</p>
<p>
<a href="https://langchain-ai.github.io/langgraphjs/reference/types/langgraph_prebuilt.CreateReactAgentParams.html"><code>stateModifier</code></a> 参数已添加到 <code>@langchain/langgraph>=0.2.27</code> 中。
<br/>
如果您使用的是旧版本，则需要使用已弃用的 <code>messageModifier</code> 参数。
<br/>
如需升级帮助，请参阅<a href="/langgraphjs/how-tos/manage-ecosystem-dependencies/">本指南</a>。
</p>
</div>

## 设置

首先，我们需要安装所需的软件包。
```bash
yarn add @langchain/langgraph @langchain/openai @langchain/core
```
本指南将使用 OpenAI 的 GPT-4o 模型。我们可以选择设置 API 密钥
对于 [LangSmith 追踪](https://smith.langchain.com/)，这将为我们提供
一流的可观测性。

In [3]:
// process.env.OPENAI_API_KEY = "sk_...";

// 可选，在 LangSmith 中添加跟踪
// process.env.LANGCHAIN_API_KEY = "ls__..."
// process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "ReAct Agent with system prompt: LangGraphJS";

ReAct Agent with system prompt: LangGraphJS


## 代码
现在我们可以使用预构建的 `createReactAgent` 函数通过系统提示来设置您的代理：

In [3]:
import { ChatOpenAI } from "@langchain/openai";
import { tool } from '@langchain/core/tools';
import { z } from 'zod';
import { createReactAgent } from "@langchain/langgraph/prebuilt";

const model = new ChatOpenAI({
    model: "gpt-4o",
  });

const getWeather = tool((input) => {
    if (input.location === 'sf') {
        return 'It\'s always sunny in sf';
    } else {
        return 'It might be cloudy in nyc';
    }
}, {
    name: 'get_weather',
    description: 'Call to get the current weather.',
    schema: z.object({
        location: z.enum(['sf','nyc']).describe("Location to get the weather for."),
    })
})

// 我们可以在这里添加我们的系统提示符
const prompt = "Respond in Italian"

const agent = createReactAgent({ llm: model, tools: [getWeather], stateModifier: prompt });

## 用法

让我们验证一下代理是否确实用意大利语回复！

In [4]:
let inputs = { messages: [{ role: "user", content: "what is the weather in NYC?" }] };
let stream = await agent.stream(inputs, {
  streamMode: "values",
});

for await (
  const { messages } of stream
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

what is the weather in NYC?
-----

[
  {
    name: 'get_weather',
    args: { location: 'nyc' },
    type: 'tool_call',
    id: 'call_PqmKDQrefHQLmGsZSSr4C7Fc'
  }
]
-----

It might be cloudy in nyc
-----

A New York potrebbe essere nuvoloso. Hai altre domande o posso aiutarti in qualcos'altro?
-----

